In [4]:
# IMPORTS
from nba_api.stats.static import players, teams
from nba_api.stats.endpoints import playercareerstats, playergamelog, teamgamelog
from nba_api.live.nba.endpoints import scoreboard, playbyplay, boxscore
from datetime import datetime, timezone
from dateutil import parser
import pandas as pd
from pprint import pprint

In [6]:
# TODAYS GAMES
board = scoreboard.ScoreBoard() # Get the scoreboard for today's games
games_data = board.games.get_dict()

# Create a list to store game information
game_list = []

# Format the game information and append it to the list
for game in games_data:
    gameTimeLTZ = parser.parse(game["gameTimeUTC"]).replace(tzinfo=timezone.utc).astimezone(tz=None)
    game_info = {
        "GameId": game['gameId'],
        "AwayTeam": game['awayTeam']['teamTricode'],
        "HomeTeam": game['homeTeam']['teamTricode'],
        "GameTimeLTZ": gameTimeLTZ
    }
    game_list.append(game_info)

# Convert the list of dictionaries to a DataFrame
games_df = pd.DataFrame(game_list)
games_df

,GameId,AwayTeam,HomeTeam,GameTimeLTZ
0,0022300733,GSW,IND,2024-02-08 19:00:00-05:00
1,0022300734,SAS,ORL,2024-02-08 19:00:00-05:00
2,0022300735,CLE,BKN,2024-02-08 19:30:00-05:00
3,0022300736,DAL,NYK,2024-02-08 19:30:00-05:00
4,0022300737,CHI,MEM,2024-02-08 20:00:00-05:00
5,0022300738,MIN,MIL,2024-02-08 20:00:00-05:00
6,0022300739,UTA,PHX,2024-02-08 21:00:00-05:00
7,0022300740,DEN,LAL,2024-02-08 22:00:00-05:00
8,0022300741,DET,POR,2024-02-08 22:00:00-05:00


In [2]:
# BOX SCORE FOR A CERTAIN GAME  
box = boxscore.BoxScore('0022300194') # Choose GameId

# Note: home_team & away_team have the identicial data structure.
players_data = box.home_team.get_dict()['players']

# Create a list of dictionaries
data_list = []
for player in players_data:
    three_pointers_attempted = player['statistics']['threePointersAttempted']
    three_pointers_made = player['statistics']['threePointersMade']

    # Calculate shooting percentage, handle the case where attempted is zero to avoid division by zero
    shooting_percentage = f"{three_pointers_made}/{three_pointers_attempted}" if three_pointers_attempted > 0 else "0/0"

    player_dict = {
        'player_id': player['personId'],
        'name': player['name'],
        'points': player['statistics']['points'],
        'rebounds': player['statistics']['reboundsTotal'],
        'assists': player['statistics']['assists'],
        '3pt %': shooting_percentage
    }
    data_list.append(player_dict)

# Create DataFrame
boxscore_df = pd.DataFrame(data_list)

# Display the DataFrame
boxscore_df

,player_id,name,points,rebounds,assists,3pt %
0,203496,Robert Covington,16,6,0,0/5
1,202699,Tobias Harris,16,7,3,1/2
2,203954,Joel Embiid,20,9,7,0/2
3,1629001,De'Anthony Melton,16,7,1,5/6
4,1630178,Tyrese Maxey,20,3,4,3/7
5,202694,Marcus Morris Sr.,0,0,0,0/2
6,201976,Patrick Beverley,0,1,1,0/2
7,1627863,Danuel House Jr.,0,0,0,0/1
8,1627788,Furkan Korkmaz,0,0,0,0/1
9,1630531,Jaden Springer,9,2,0,1/1
